In [2]:
import os
import pandas as pd
import re

In [5]:
mobility_path = '../data/raw/'
processed_path = '../data/processed/'
raw_path = '../data/raw/'


In [119]:
def update_mobility(processed:pd.DataFrame, mobility:pd.DataFrame):
    
    '''
    processed: old time_series data
    mobility: new mobility csv from Google
    '''
    
    # clean mobility data
    USA = mobility.loc[mobility['country_region'] == 'United States'].reset_index()
    to_drop = ['index', 'country_region_code', 'country_region']
    USA = USA.drop(to_drop, axis=1)
    USA.insert(1, 'county', USA['sub_region_2'].str.lower())
    USA = USA.drop(['sub_region_2'],axis=1)
    USA['county'].loc[(USA['sub_region_1'] == 'District of Columbia')] = 'District of Columbia'
    USA = USA.loc[~pd.isna(USA['county'])]
    USA['county'] = USA['county'].map(lambda x: re.sub(' county| city','', x)).str.lower()
    USA.insert(1, 'state', USA['sub_region_1'].str.lower())
    USA = USA.drop(['sub_region_1'],axis=1)
    
    # prepare old time series data
    processed['county'] = processed['county'].str.lower()
    processed['state'] = processed['state'].str.lower()
    
    # take only subset of counties according to proccessed data
    counties = pd.unique(processed['county'].str.lower().map(lambda x: re.sub(' city','', x)))
    USA = USA.loc[USA['county'].str.lower().isin(counties)]
    
    # deal with duplicates
    columns = set(USA.columns) - set(['state', 'county', 'date'])
    agg = {key:'mean' for key in columns}
    USA_grouped = USA.groupby(['state', 'county', 'date']).aggregate(agg)
    
    df = pd.merge(processed,USA_grouped, on=['date','county', 'state'], how='left')
    
    return df

In [120]:
df = update_mobility(processed, mobility)

In [65]:
mobility = pd.read_csv(mobility_path+'Global_Mobility_Report.csv', low_memory=False)
processed = pd.read_csv(processed_path + 'time_series.csv')
zips = pd.read_csv(raw_path + 'uszips.csv')

In [10]:
# take only subset of counties according to proccessed data
counties = pd.unique(processed['county'].str.lower().map(lambda x: re.sub(' city','', x)))
USA = USA.loc[USA['county'].str.lower().isin(counties)]

In [11]:
# difference in counties
difference = set(pd.unique(counties)) - set(USA['county'].str.lower().unique())

In [80]:
# Cut time
# USA = USA.loc[USA['date']<'2020-04-23']

In [81]:
# deal with duplicates
columns = set(USA.columns) - set(['state', 'county', 'date'])
agg = {key:'mean' for key in columns}
USA_grouped = USA.groupby(['state', 'county', 'date']).aggregate(agg)

In [82]:
df = pd.merge(processed,USA_grouped, on=['date','county', 'state'], how='left')

In [129]:
df.to_csv(processed_path + 'time_series_mobility.csv',index=False)
# df = pd.read_csv(processed_path + 'time_series_mobility.csv')

In [130]:
df

,date,county,state,fips,cases,deaths,cldCvrMin,cldCvrAvg,cldCvrMax,dewPtMin,...,wetBulbAvg,wetBulbMax,first_date,days_since_10_cases,residential_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,transit_stations_percent_change_from_baseline,retail_and_recreation_percent_change_from_baseline,parks_percent_change_from_baseline,workplaces_percent_change_from_baseline
0,2020-01-21,snohomish,washington,53061,1,0,7.0,71.0,100.0,37.4,...,42.7,44.1,2020-03-05,-44,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-22,snohomish,washington,53061,1,0,31.0,74.0,100.0,39.9,...,42.4,44.6,2020-03-05,-43,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-23,snohomish,washington,53061,1,0,75.0,93.0,100.0,44.3,...,48.6,50.7,2020-03-05,-42,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-24,snohomish,washington,53061,1,0,15.0,57.0,95.0,43.9,...,46.7,48.3,2020-03-05,-41,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-25,snohomish,washington,53061,1,0,17.0,72.0,100.0,41.4,...,44.5,48.0,2020-03-05,-40,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54946,2020-04-15,saline,nebraska,31151,1,0,0.0,1.0,12.0,17.4,...,33.8,42.8,2020-04-21,-6,NaN,NaN,NaN,NaN,NaN,NaN
54947,2020-04-16,saline,nebraska,31151,3,0,45.0,88.0,100.0,22.7,...,32.3,34.5,2020-04-21,-5,NaN,NaN,NaN,NaN,NaN,NaN
54948,2020-04-17,saline,nebraska,31151,4,0,0.0,35.0,100.0,23.9,...,33.5,40.9,2020-04-21,-4,NaN,NaN,NaN,NaN,NaN,NaN
54949,2020-04-18,saline,nebraska,31151,6,0,0.0,9.0,69.0,30.1,...,45.3,55.4,2020-04-21,-3,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
mobility = pd.read_csv(mobility_file)
agg_df = update_mobility(agg_df, mobility)

income = pd.read_csv(income_file)
agg_df = add_income(agg_df, income)

population = pd.read_csv(population_file)
agg_df = add_population(agg_df, population)